In [81]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import importlib
import datetime

import shioaji as sj
import login.shioaji_login as shioaji_login
from shioaji import TickFOPv1, BidAskFOPv1, Exchange
importlib.reload(shioaji_login)

lastcontractprice={}
TXO_weekly_dict={}

In [82]:
def place_cb(stat, msg):
    """
    Called every time an order or a deal has been detected.
    
    :global param: msg_list ()

    return: none
    """
    print('my_place_callback')

In [83]:
def get_next_wednesday(today):
    """
    Get next week's number in this month.
    
    :param: today (datetime.date)

    :return: nextwed (int)
    """
    oneday = datetime.timedelta(days = 1)
    while today.weekday() != 2: # 2 mean Wednesday
        today += oneday
    nextwed = today
    return nextwed

In [84]:
def get_opt_TXnum(date):
    """
    Get prefix(TX?) symbol code for today's week option.
    
    :param: date (datetime.date)

    :return: res (str)
    """
    endday=get_next_wednesday(date)
    beginday=get_next_wednesday(datetime.date(endday.year, endday.month, 1))
    begin = int(beginday.strftime("%W"))
    end = int(endday.strftime("%W"))

    week=end - begin + 1 
    if week == 3: # no W3, instead is month expiration
        res='TXO'
    else:
        res='TX'+str(week)
    return res

In [85]:
def get_option_code(cp):
    """
    Get final two symbol code for whole week option.
    
    :param: cp (str)

    :return: option_code (str)
    """
    if(datetime.datetime.now().weekday()!=2): # not wednesday
        now = get_next_wednesday(datetime.datetime.now())
    else:
        now = datetime.datetime.now()
    month = now.month
    year = now.year

    month_to_code = '0ABCDEFGHIJKL'
    if cp=="P":
        month_to_code = '0MNOPQRSTUVWX'

    option_code = month_to_code[month]
    option_code += str(year%10)
    
    return option_code

In [86]:
def makebound(input,cbr):
    """
    Get every Option strike price, input is market now, and cbr is gap from each strike price.
    
    :param: input (int)
    :param: cbr (int)
    :return:  (tuple)
    """
    c='['+str(cbr*int(input/cbr))+'-'+str(cbr*int(input/cbr)+cbr)+')'
    return (cbr*int(input/cbr),str(cbr*int(input/cbr)+cbr),c)

In [87]:
def get_tse_bound(upper,bottom,txname):
    """
    Get all strike prices base on upper and bottom price.
    
    :global param: api
    :param: upper (int)
    :param: bottom (int)
    :return:  (tuple): (int, str, str)
    """
    global api

    putcode=get_option_code("P")
    callcode=get_option_code("C")

    contract_snap = api.snapshots([api.Contracts.Indexs.TSE.TSE001])[0]
    close=contract_snap.close # get TSE001's market price
    upc=close+upper
    botc=close+bottom 
    bt=makebound(botc,50)[0]
    up=makebound(upc,50)[0]
    contract_code_list=[ txname+str(_)+putcode for _ in range(bt,up+50,50)]
    contract_code_list+=[ txname+str(_)+callcode for _ in range(bt,up+50,50)]
    return contract_code_list

In [88]:
def getsnapOptions():
    """
    Get all snap options.
    
    :global param: lastcontractprice
    :global param: api
    :return:  None
    """
    global api
    global lastcontractprice

    contractname=get_opt_TXnum(datetime.datetime.today())
    contracts=api.Contracts.Options[contractname] # get whole contract available on market
    code_list=get_tse_bound(2000,-2000,contractname) # get specifc contracts depend on parameter limit
    
    for contract in contracts:
        if contract.code in code_list:
            try:
                contract_snap = api.snapshots([contract])[0]
                if TXO_weekly_dict.get(contract.code[:-2]) is None:
                    TXO_weekly_dict[contract.code[:-2]] = {'C':None,'P':None}
                data={
                    'askprice':contract_snap.sell_price, #賣方第一檔
                    'bidprice':contract_snap.buy_price, #買方第一檔
                    'close':contract_snap.close,
                    'snap': contract_snap,
                    'contract':contract # 商品合約物件
                }
                TXO_weekly_dict[contract.code[:-2]][contract.symbol[-1]]=data
                lastcontractprice[contract.code]=data
            except Exception as e:
                print(e)
                print("error")

In [89]:
def main():
    """
    Controlled whole program, make app execute.
    
    :global param: api
    """
    # log in
    global api
    api = shioaji_login.login()
    api.set_order_callback(place_cb)

    getsnapOptions()

    # print(api.Contracts.Options.TX4)



In [90]:
if __name__ == "__main__":
    api = None
    main() 

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
Login with ID R124743829
Login status: [FutureAccount(person_id='R124743829', broker_id='F002000', account_id='1697256', signed=True, username='劉庭銘'), Account(account_type=<AccountType.H: 'H'>, person_id='R124743829', broker_id='9A9X', account_id='0150198', username='劉庭銘\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000'), StockAccount(person_id='R124743829', broker_id='9A9X', account_id='0510980', signed=True, username='劉庭銘\u3000\u3000')]
Activating CA at the path C:/sinopac/Sinopac.pfx


code list ['TX412600W2', 'TX412650W2', 'TX412700W2', 'TX412750W2', 'TX412800W2', 'TX412850W2', 'TX412900W2', 'TX412950W2', 'TX413000W2', 'TX413050W2', 'TX413100W2', 'TX413150W2', 'TX413200W2', 'TX413250W2', 'TX413300W2', 'TX413350W2', 'TX413400W2', 'TX413450W2', 'TX41